<a href="https://colab.research.google.com/github/codekayenat/Agent-deployed-on-Agent-Engine/blob/main/Deploy_AI_Agent_On_Agent_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Vertex AI SDK

In [ ]:
pip install google-cloud-aiplatform[adk,agent_engines]

# Authenticate (Colab only)

In [ ]:
from google.colab import auth
auth.authenticate_user()

# Initialize Vertex AI SDK

In [ ]:
import vertexai

PROJECT_ID = "your-project-id"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://your-google-cloud-storage-bucket"

vertexai.init(
    project= "deploy-agent-on-agent-engine",
    location= "us-central1",
    staging_bucket= "gs://deploy_agent_on_agent_engine",
)

# Define the Agent using ADK

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search

#Define the agent using ADK
root_agent = Agent(
    name="basic_search_agent", #name of your agent
    model="gemini-2.5-flash-preview-04-17", # specify the model version
    description=(
        "Agent to answer questions using google search." # Brief description of the agent
    ),
    instruction=(
        "You are a helpful agent who can answer questions using internet. Just ask me anything"
    ),
    tools=[google_search], # use the Pre-built Google search tool
    output_key= "google_search" # key where search result will be stored
)

# Prepare Agent for Deployment AdkApp

In [ ]:
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

# Create a Local Test Session

In [ ]:
session = app.create_session(user_id="u_123")  # Replace with your desired user ID
session

Session(id='e475c811-8d4e-4e5c-9161-a7148949f6c6', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749966679.013655)

# List Local Test Sessions

In [ ]:
active_sessions= app.list_sessions(user_id="u_123")  # Replace with the same user ID used earlier
active_sessions

ListSessionsResponse(sessions=[Session(id='e475c811-8d4e-4e5c-9161-a7148949f6c6', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749966679.013655)])

# Get a Specific Local Test Session

In [ ]:
specific_session = app.get_session(user_id="u_123", session_id=session.id)
specific_session

Session(id='e475c811-8d4e-4e5c-9161-a7148949f6c6', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749966679.013655)

In [ ]:
pip install pydantic cloudpickle

In [ ]:
pip show pydantic cloudpickle

Name: pydantic
Version: 2.11.5
Summary: Data validation using Python type hints
Home-page: https://github.com/pydantic/pydantic
Author: 
Author-email: Samuel Colvin <s@muelcolvin.com>, Eric Jolibois <em.jolibois@gmail.com>, Hasan Ramezani <hasan.r67@gmail.com>, Adrian Garcia Badaracco <1755071+adriangb@users.noreply.github.com>, Terrence Dorsey <terry@pydantic.dev>, David Montague <david@pydantic.dev>, Serge Matveenko <lig@countzero.co>, Marcelo Trylesinski <marcelotryle@gmail.com>, Sydney Runkle <sydneymarierunkle@gmail.com>, David Hewitt <mail@davidhewitt.io>, Alex Hall <alex.mojaki@gmail.com>, Victorien Plot <contact@vctrn.dev>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: annotated-types, pydantic-core, typing-extensions, typing-inspection
Required-by: albumentations, confection, fastapi, google-adk, google-cloud-aiplatform, google-genai, google-generativeai, gradio, langchain, langchain-core, langsmith, mcp, openai, pydantic-settings, spacy, thinc, wandb, w

# Send Query to Local Agent

In [ ]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="whats the weather in new york",
):
 print(event)

Exception in thread Thread-11 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 139, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 127, in _invoke_run_async
    async for event in self.run_async(
  File "/u

# Deploy Agent to Agent Engine

In [ ]:
from vertexai import agent_engines

remote_app = agent_engines.create(
    agent_engine=app,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"
    ]
)

INFO:vertexai.agent_engines:Identified the following requirements: {'cloudpickle': '3.1.1', 'pydantic': '2.11.5', 'google-cloud-aiplatform': '1.95.1'}
INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.11.5', 'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'pydantic==2.11.5', 'cloudpickle==3.1.1']
INFO:vertexai.agent_engines:Using bucket deploy_agent_on_agent_engine
INFO:vertexai.agent_engines:Wrote to gs://deploy_agent_on_agent_engine/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://deploy_agent_on_agent_engine/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://deploy_agent_on_agent_engine/agent_engine/dependencies.tar.gz


PermissionDenied: 403 Vertex AI API has not been used in project deploy-agent-on-agent-engine before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/aiplatform.googleapis.com/overview?project=deploy-agent-on-agent-engine then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. [reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "serviceTitle"
  value: "Vertex AI API"
}
metadata {
  key: "containerInfo"
  value: "deploy-agent-on-agent-engine"
}
metadata {
  key: "consumer"
  value: "projects/deploy-agent-on-agent-engine"
}
metadata {
  key: "activationUrl"
  value: "https://console.developers.google.com/apis/api/aiplatform.googleapis.com/overview?project=deploy-agent-on-agent-engine"
}
, locale: "en-US"
message: "Vertex AI API has not been used in project deploy-agent-on-agent-engine before or it is di

# Create a Remote Session on Deployed Agent

In [ ]:
remote_session = remote_app.create_session(user_id="u_456")
remote_session

NameError: name 'remote_app' is not defined

# List Remote Session

In [ ]:
remote_active_sessions = remote_app.list_sessions(user_id="456")
remote_active_sessions

# Get a Specific Remote Session

In [ ]:
remote_app.get_session(user_id="u_456", session_id=remote_session["id"])

# Send Query to Deployed Agent

In [ ]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="whats the weather in calgary, Canada",
):
    print(event)

# Retrieve an Existing Deployed Agent